In [27]:
!pip install rdflib


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [28]:
import numpy as np
import pandas as pd

import os
import networkx as nx
from matplotlib import pyplot as plt
from rdflib import Graph
from rdflib.plugins.sparql import prepareQuery

#### Setup

In [29]:
import pandas as pd
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS, XSD, DCTERMS
from urllib.parse import quote

# Define the namespaces based on your URIs
CKO = Namespace("https://w3id.org/countrykg/ontology/")
SCHEMA = Namespace("https://schema.org/")
WD = Namespace("https://www.wikidata.org/wiki/")
WDP = Namespace("https://www.wikidata.org/wiki/Property:")
TIME = Namespace("https://www.w3.org/2006/time#")
DCTYPES = Namespace("http://purl.org/dc/terms/")
DBO = Namespace("http://dbpedia.org/ontology/")

# Create a new graph
ontology_g = Graph()

# Bind the namespace prefixes for more readable output
ontology_g.bind("cko", CKO)
ontology_g.bind("schema", SCHEMA)
ontology_g.bind("wd", WD)
ontology_g.bind("wdp", WDP)
ontology_g.bind("time", TIME)
ontology_g.bind("dctypes", DCTYPES)
ontology_g.bind("rdfs", RDFS)
ontology_g.bind("dbo", DBO)


## Create Schema

#### Add Classes

In [30]:
classes_dict ={
    
    'Artist':'wd:Q639669'
    ,'Family':'wd:Q8436'
    ,'PublicImage':'wd:Q6738447'
    ,'Label':'wd:Q18127'
    ,'Song':'wd:Q7366'
    ,'BillboardSpot':'schema:BillboardSpot' # didn't find real class
    ,'Ranking':'wd:Q4120621'
    ,'Country':'wd:Q6256'
    ,'Male':'wd:Q6581097'
    ,'Female':'wd:Q6581072'
}
classes_dict_uri = {
    'Artist': WD.Q639669,
    'Family': WD.Q8436,
    'PublicImage': WD.Q6738447,
    'Label': WD.Q18127,
    'Song': WD.Q7366,
    'BillboardSpot': SCHEMA.BillboardSpot, # Note: Placeholder as the real class wasn't found
    'Ranking': WD.Q4120621,
    'Country': WD.Q6256,
    'Male': WD.Q6581097,
    'Female': WD.Q6581072
}


In [31]:
# Bind common prefixes for readability
ontology_g.bind("wd", "http://www.wikidata.org/entity/")
ontology_g.bind("rdfs", "http://www.w3.org/2000/01/rdf-schema#")

# Manually adding each triple with Literal(..) for labels and specifying language as 'en'

# Artist
ontology_g.add((WD.Q639669, RDF.type, RDFS.Class))
ontology_g.add((WD.Q639669, RDFS.label, Literal('Artist', lang='en')))

# Family
ontology_g.add((WD.Q8436, RDF.type, RDFS.Class))
ontology_g.add((WD.Q8436, RDFS.label, Literal('Family', lang='en')))

# PublicImage
ontology_g.add((WD.Q6738447, RDF.type, RDFS.Class))
ontology_g.add((WD.Q6738447, RDFS.label, Literal('PublicImage', lang='en')))

# Label
ontology_g.add((WD.Q18127, RDF.type, RDFS.Class))
ontology_g.add((WD.Q18127, RDFS.label, Literal('Label', lang='en')))

# Song
ontology_g.add((WD.Q7366, RDF.type, RDFS.Class))
ontology_g.add((WD.Q7366, RDFS.label, Literal('Song', lang='en')))

# Ranking
ontology_g.add((WD.Q4120621, RDF.type, RDFS.Class))
ontology_g.add((WD.Q4120621, RDFS.label, Literal('Ranking', lang='en')))

# Country
ontology_g.add((WD.Q6256, RDF.type, RDFS.Class))
ontology_g.add((WD.Q6256, RDFS.label, Literal('Country', lang='en')))

# Male
ontology_g.add((WD.Q6581097, RDF.type, RDFS.Class))
ontology_g.add((WD.Q6581097, RDFS.label, Literal('Male', lang='en')))

# Female
ontology_g.add((WD.Q6581072, RDF.type, RDFS.Class))
ontology_g.add((WD.Q6581072, RDFS.label, Literal('Female', lang='en')))

#ontology_g.add((SCHEMA.BillboardSpot, RDF.type, RDFS.Class)) # BillboardSpot
#ontology_g.add((SCHEMA.BillboardSpot, RDFS.label, 'BillboardSpot'))

<Graph identifier=N1016582c0ea44b008880e3cab6f580b9 (<class 'rdflib.graph.Graph'>)>


#### Add Properties

In [32]:
properties_dict = {
    'hasCharacteristic':'Wp:P1552'
    ,'record label':'wdp:P264'
    ,'performer':'wdp:P175'
    ,'ranking':'wdp:P1352'
    ,'isFrom':'wdp:P27'
    ,'gender':'wdp:P21'
    ,'genre': 'wdp.P136'
    ,'text':'schema:text'
    ,'name':'rdfs:label'
    ,'datePosted':'schema:datePosted'
    ,'value':'schema:value'
    ,'publication date':'wdp:P577'
    ,'hasLifeTimeSales':'schema:hasLifeTimeSales'
    ,'score':'wdp:P444'
    ,'number of children':'wdp.P1971'
}
properties_dict_uri = {
    'hasCharacteristic': WDP.P1552,
    'record label': WDP.P264,
    'performer': WDP.P175,
    'ranking': WDP.P1352,
    'isFrom': WDP.P27,
    'genre': WDP.P136,
    'gender': WDP.P21,
    'text': SCHEMA.text,
    'name': RDFS.label,
    'datePosted': SCHEMA.datePosted,
    'value': SCHEMA.value,
    'publication date': WDP.P577,
    'hasLifeTimeSales': SCHEMA.hasLifeTimeSales
    ,'score': WDP.P444
    ,'number of children':WDP.P1971
    ,'hasTwitterFollowers':SCHEMA.hasTwitterFollowers
    ,'hasYoutubeFollowers':SCHEMA.hasYoutubeFollowers
}


In [33]:

ontology_g.add((WDP.P1552, RDF.type, RDF.Property))  #hasCharacteristic
ontology_g.add((WDP.P1552, RDFS.label, Literal('hasCharacteristic', lang='en')))
ontology_g.add((WDP.P1552, RDFS.domain, WD.Q639669))  
ontology_g.add((WDP.P1552, RDFS.range, WD.Q8436)) 
ontology_g.add((WDP.P1552, RDFS.range, WD.Q6738447)) 

ontology_g.add((WDP.P264, RDF.type, RDF.Property))  #record label
ontology_g.add((WDP.P264, RDFS.label, Literal('record label', lang='en')))
ontology_g.add((WDP.P264, RDFS.domain, WD.Q639669))  
ontology_g.add((WDP.P264, RDFS.range, WD.Q18127))  

ontology_g.add((WDP.P175, RDF.type, RDF.Property))  #performer
ontology_g.add((WDP.P175, RDFS.label, Literal('performer', lang='en')))
ontology_g.add((WDP.P175, RDFS.domain, WD.Q7366))  
ontology_g.add((WDP.P175, RDFS.range, WD.Q639669))  

ontology_g.add((WDP.P1352, RDF.type, RDF.Property))  #ranking
ontology_g.add((WDP.P1352, RDFS.label, Literal('ranking', lang='en')))
ontology_g.add((WDP.P1352, RDFS.domain, WD.Q7366))  
ontology_g.add((WDP.P1352, RDFS.range, WD.Q4120621))

ontology_g.add((WDP.P27, RDF.type, RDF.Property))  #isFrom
ontology_g.add((WDP.P27, RDFS.label, Literal('isFrom', lang='en')))
ontology_g.add((WDP.P27, RDFS.domain, WD.Q639669))
ontology_g.add((WDP.P27, RDFS.range, WD.Q6256))  #Country


ontology_g.add((WDP.P21, RDF.type, RDF.Property))  #gender
ontology_g.add((WDP.P21, RDFS.label, Literal('gender', lang='en')))
ontology_g.add((WDP.P21, RDFS.domain, WD.Q639669))  #Artist
ontology_g.add((WDP.P21, RDFS.range, WD.Q6581097))  #Male
ontology_g.add((WDP.P21, RDFS.range, WD.Q6581072))  #Female

ontology_g.add((SCHEMA.text, RDF.type, RDF.Property))  #text
ontology_g.add((SCHEMA.text, RDFS.label, Literal('text', lang='en')))
ontology_g.add((SCHEMA.text, RDFS.domain, WD.Q6738447))
ontology_g.add((SCHEMA.text, RDFS.range, RDF.langString))


ontology_g.add((RDFS.label, RDF.type, RDF.Property))  #name
ontology_g.add((RDFS.label, RDFS.label, Literal('label', lang='en')))
ontology_g.add((RDFS.label, RDFS.range, RDF.langString))

ontology_g.add((SCHEMA.datePosted, RDF.type, RDF.Property))  #datePosted
ontology_g.add((SCHEMA.datePosted, RDFS.label, Literal('datePosted', lang='en')))
ontology_g.add((SCHEMA.datePosted, RDFS.domain, WD.Q4120621))
ontology_g.add((SCHEMA.datePosted, RDFS.range, XSD.date))


ontology_g.add((SCHEMA.value, RDF.type, RDF.Property))  #value
ontology_g.add((SCHEMA.value, RDFS.label, Literal('value', lang='en')))
ontology_g.add((SCHEMA.value, RDFS.domain, WD.Q4120621))
ontology_g.add((SCHEMA.value, RDFS.range, XSD.integer))


ontology_g.add((WDP.P577, RDF.type, RDF.Property))  #publication date
ontology_g.add((WDP.P577, RDFS.label, Literal('publication date', lang='en')))
ontology_g.add((WDP.P577, RDFS.domain, WD.Q7366))
ontology_g.add((WDP.P577, RDFS.range, XSD.date))


ontology_g.add((SCHEMA.hasLifeTimeSales, RDF.type, RDF.Property))  #hasLifeTimeSales
ontology_g.add((SCHEMA.hasLifeTimeSales, RDFS.label, Literal('hasLifeTimeSales', lang='en')))
ontology_g.add((SCHEMA.hasLifeTimeSales, RDFS.domain, WD.Q639669))
ontology_g.add((SCHEMA.hasLifeTimeSales, RDFS.range, XSD.double))


ontology_g.add((WDP.P136, RDF.type, RDF.Property))  #genre
ontology_g.add((WDP.P136, RDFS.label, Literal('genre', lang='en')))
ontology_g.add((WDP.P136, RDFS.domain, WD.Q639669))
ontology_g.add((WDP.P136, RDFS.range, RDF.langString
))


ontology_g.add((WDP.P444, RDF.type, RDF.Property))  #score
ontology_g.add((WDP.P444, RDFS.label, Literal('score', lang='en')))
ontology_g.add((WDP.P444, RDFS.domain, WD.Q6738447))
ontology_g.add((WDP.P444, RDFS.range, XSD.double))


ontology_g.add((WDP.P1971, RDF.type, RDF.Property))  #number of children
ontology_g.add((WDP.P1971, RDFS.label, Literal('number of children (-1 if not available)', lang='en')))

ontology_g.add((WDP.P1971, RDFS.domain, WD.Q639669))
ontology_g.add((WDP.P1971, RDFS.range, XSD.integer))


ontology_g.add((SCHEMA.hasTwitterFollowers, RDF.type, RDF.Property))  #hasTwitterFollowers 
ontology_g.add((SCHEMA.hasTwitterFollowers, RDFS.label, Literal('hasTwitterFollowers', lang='en')))
ontology_g.add((SCHEMA.hasTwitterFollowers, RDFS.domain, WD.Q639669))

ontology_g.add((SCHEMA.hasYoutubeFollowers, RDF.type, RDF.Property))  #hasYoutubeFollowers
ontology_g.add((SCHEMA.hasYoutubeFollowers, RDFS.label, Literal('hasYoutubeFollowers', lang='en')))
ontology_g.add((SCHEMA.hasYoutubeFollowers, RDFS.domain, WD.Q639669))

<Graph identifier=N1016582c0ea44b008880e3cab6f580b9 (<class 'rdflib.graph.Graph'>)>

In [34]:
ontology_g.serialize('KnowledgeGraphs//ontology_graph.ttl', format='turtle')

<Graph identifier=N1016582c0ea44b008880e3cab6f580b9 (<class 'rdflib.graph.Graph'>)>

## Knowledge Graph Construction

##### Import the preprocessed data%

In [35]:
import pandas as pd

# Create a new graph
g = Graph()

# Bind the namespace prefixes for more readable output
g.bind("cko", CKO)
g.bind("schema", SCHEMA)
g.bind("wd", WD)
g.bind("wdp", WDP)
g.bind("time", TIME)
g.bind("dctypes", DCTYPES)
g.bind("rdfs", RDFS)
g.bind("dbo", DBO)


In [36]:
sales_df = pd.read_csv('Data//best_selling_artists.csv')
quantity_df = pd.read_csv('Data//celebrity_quantity(children&followers).csv')
relatives_label_df = pd.read_csv('Data//celebrity_relatives&label.csv') 
charts10_df = pd.read_csv('Data//charts10.csv')
artist_public_image_df = pd.read_csv('Data//artist_public_image.csv')
quantity_df.rename(columns={'Unnamed: 0':'Artist'}, inplace=True)

C:\Users\danpa\AppData\Local\Temp\ipykernel_39072\670551846.py:4: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  charts10_df = pd.read_csv('Data//charts10.csv')


In [37]:
relatives_label_df.loc[relatives_label_df['Artist'] == 'Madonna']

,Artist,Children,Spouse,Siblings,RecordLabel
5,Madonna,"['Jesus', 'James the Just', 'Simon, brother of...",['Joseph'],"['Salome', 'Mary of Clopas']",[]


In [38]:
sales_df

,Artist,Country,period_active,Year,Genre,TCU,Sales
0,The Beatles,United Kingdom,1960–1970,1962,Rock / pop,290.4 million,600 million500 million
1,Elvis Presley,United States,1954–1977,1956,Rock and roll / pop / country,231.2 million,500 million
2,Michael Jackson,United States,1964–2009,1971,Pop / rock / dance / soul / R&B,277.3 million,400 million
3,Elton John,United Kingdom,1964–present,1970,Pop / rock,208 million,300 million250 million
4,Queen,United Kingdom,1971–present,1973,Rock,184 million,300 million250 million
...,...,...,...,...,...,...,...
116,The Police,United Kingdom,1977–19862007–2008,1978,Pop / rock,42.2 million,75 million
117,Gloria Estefan,United States,1975–present,1984,Latin pop / dance pop,37.6 million,75 million
118,Barry Manilow,United States,1973–present,1973,Pop / soft rock,36.7 million,75 million
119,Kiss,United States,1972–present,1974,Hard rock / heavy metal,28.7 million,75 million


In [39]:
## Add Artist
for i in range(len(sales_df)):
    artist = URIRef(WD + quote(sales_df['Artist'][i]))
    g.add((artist, RDF.type, classes_dict_uri['Artist']))
    g.add((artist, RDFS.label, Literal(sales_df['Artist'][i], lang='en')))
    
    g.add((artist, properties_dict_uri['hasLifeTimeSales'], Literal(float(sales_df['TCU'][i].split()[0]),datatype=XSD.double)))
    g.add((artist,properties_dict_uri['genre'], Literal(sales_df['Genre'][i], lang='en')))     
    

In [40]:
# add madonna label
madonna_uri = URIRef(WD + quote('Madonna'))
g.add((madonna_uri, RDFS.label, Literal('Madonna', lang='en')))

<Graph identifier=Nf44a790461fb425396682f633a909df1 (<class 'rdflib.graph.Graph'>)>

In [41]:
## Add Public Image
        
for idx,row in artist_public_image_df.iterrows():
    artist = URIRef(WD + quote(row['Artist']))
    public_image = URIRef(WD + quote(row['Public Image']) + '#' + quote(row['Artist']))
    
    
    g.add((public_image, RDF.type, classes_dict_uri['PublicImage']))
    g.add((public_image, RDFS.label, Literal(row['Public Image'], lang='en')))
    
    g.add((artist, properties_dict_uri['hasCharacteristic'], public_image))
    g.add((public_image, properties_dict_uri['score'], Literal(row['Public Image Score'],datatype=XSD.double)))

In [42]:
## add country of artist
country_occurrence = []
for i in range(len(sales_df)):
    country_uri = URIRef(WD + quote(sales_df['Country'][i]))
    
    if sales_df['Country'][i] not in country_occurrence:
        g.add((country_uri, RDF.type, classes_dict_uri['Country']))
        g.add((country_uri, RDFS.label, Literal(sales_df['Country'][i], lang='en')))
        country_occurrence.append(sales_df['Country'][i])

    artist_uri = URIRef(WD + quote(sales_df['Artist'][i]))
    g.add((artist_uri, properties_dict_uri['isFrom'], country_uri))

In [43]:
import re


# add number of children
for idx, row in quantity_df.iterrows():
    artist = URIRef(WD + quote(row['Artist']))
    family = URIRef(WD + 'Family' + '#' + quote(row['Artist']))
    
    g.add((family, RDF.type, classes_dict_uri['Family']))
    g.add((family, RDFS.label, Literal(('Family of '+ row['Artist']), lang='en')))
    
    
    res = re.search(r'\d+', row['numberOfChildren'])
    if res:
        number_of_children = int(res.group())
        g.add((family, properties_dict_uri['number of children'], Literal(number_of_children,datatype=XSD.integer)))


    g.add((artist, properties_dict_uri['hasCharacteristic'], family))

In [44]:
# add label as an object, for future extensions
label_occurrence = []


for i in range(len(relatives_label_df)):
    # Split the label on commas to work with each label separately
    if  relatives_label_df['RecordLabel'][i] == '[]': continue
    
    
    labels = relatives_label_df['RecordLabel'][i].split(',')
    artist = relatives_label_df['Artist'][i]
    
    
    labels[0] = labels[0][1:]
    labels[-1] = labels[-1][:-1]
    
    for label in labels:
        label_uri = URIRef(WD + quote(label) + '#' + quote(artist))   
        
        # Check if the label is already in the list
        if label not in label_occurrence:
           label_occurrence.append(label)
        
    
           g.add((label_uri, RDF.type, classes_dict_uri['Label']))
           g.add((label_uri, RDFS.label, Literal(label, lang='en'))) 
           
        artist_uri = URIRef(WD + quote(artist))
        print(label_uri)
        g.add((artist_uri, properties_dict_uri['record label'], label_uri))
    

https://www.wikidata.org/wiki/%27Apple%20Records%27#The%20Beatles
https://www.wikidata.org/wiki/%20%27Capitol%20Records%27#The%20Beatles
https://www.wikidata.org/wiki/%20%27Parlophone%27#The%20Beatles
https://www.wikidata.org/wiki/%20%27United%20Artists%20Records%27#The%20Beatles
https://www.wikidata.org/wiki/%20%27Vee-Jay%20Records%27#The%20Beatles
https://www.wikidata.org/wiki/%20%27Polydor%20Records%27#The%20Beatles
https://www.wikidata.org/wiki/%20%27Parlophon%27#The%20Beatles
https://www.wikidata.org/wiki/%20%27I%20Dischi%20dello%20Zodiaco%27#The%20Beatles
https://www.wikidata.org/wiki/%27Sun%20Records%27#Elvis%20Presley
https://www.wikidata.org/wiki/%20%27RCA%20Victor%27#Elvis%20Presley
https://www.wikidata.org/wiki/%27Epic%20Records%27#Michael%20Jackson
https://www.wikidata.org/wiki/%20%27Sony%20Music%27#Michael%20Jackson
https://www.wikidata.org/wiki/%20%27Motown%27#Michael%20Jackson
https://www.wikidata.org/wiki/%20%27Universal%20Music%20Group%27#Michael%20Jackson
https://www.

In [45]:
label_occurrence    


["'Apple Records'",
 " 'Capitol Records'",
 " 'Parlophone'",
 " 'United Artists Records'",
 " 'Vee-Jay Records'",
 " 'Polydor Records'",
 " 'Parlophon'",
 " 'I Dischi dello Zodiaco'",
 "'Sun Records'",
 " 'RCA Victor'",
 "'Epic Records'",
 " 'Sony Music'",
 " 'Motown'",
 " 'Universal Music Group'",
 " 'Sony BMG'",
 " 'Legacy Recordings'",
 " 'Steeltown Records'",
 "'Universal Records'",
 " 'Island Records'",
 " 'Philips Records'",
 " 'DJM Records'",
 " 'Uni'",
 " 'Geffen Records'",
 " 'Paramount Records'",
 " 'Mercury Records'",
 " 'MCA Records'",
 " 'Def Jam Recordings'",
 " 'Congress'",
 " 'The Rocket Record Company'",
 " 'IL'",
 " 'Chrysalis Records'",
 " 'A&M Records'",
 " 'Regal Zonophone'",
 " 'Stateside Records'",
 " 'Cube Records'",
 "'Elektra'",
 " 'EMI Records'",
 " 'Hollywood Records'",
 " 'EMI'",
 "'Atlantic Records'",
 " 'Decca'",
 " 'Swan Song Records'",
 "'Def Jam Recordings'",
 " 'Westbury Road Entertainment'",
 " 'Roc Nation'",
 "'Tower Records'",
 " 'Harvest'",
 " 'Co

In [46]:
relatives_label_df.loc[relatives_label_df['Artist'] == 'Madonna']

,Artist,Children,Spouse,Siblings,RecordLabel
5,Madonna,"['Jesus', 'James the Just', 'Simon, brother of...",['Joseph'],"['Salome', 'Mary of Clopas']",[]


In [47]:
# add each song
for song in charts10_df['song'].unique():
    song_uri = URIRef(WD + quote(song) + '#' + 'song')
    g.add((song_uri, RDF.type, classes_dict_uri['Song']))
    g.add((song_uri, RDFS.label, Literal(song, lang='en')))

In [48]:
import math

# add performers of each song
song_occurrence = []
for idx,row in charts10_df.iterrows():
    if row['song'] not in song_occurrence:   
        song_occurrence.append(row['song'])
        song_uri = URIRef(WD + quote(row['song']) + '#' + 'song')
        artists = [row['Artist1'], row['Artist2'], row['Artist3'], row['Artist4'], row['Artist5']] 
     
        for value in  [value for value in artists if isinstance(value, str)]:
            artist_uri = URIRef(WD + quote(value))
            g.add((song_uri, properties_dict_uri['performer'], artist_uri))

In [49]:
# add ranking of each song
for idx,row in charts10_df.iterrows():
    song_uri = URIRef(WD + quote(row['song']) + '#' + 'song')
    ranking_uri = URIRef(WD +  '#' + quote(row['song']) + '#' +  quote(row['Artist']) + '#' + str(idx) + '#ranking')
    
    g.add((song_uri, properties_dict_uri['ranking'], ranking_uri))
    
    
    
    g.add((ranking_uri, RDF.type, classes_dict_uri['Ranking']))
    g.add((ranking_uri, properties_dict_uri['value'], Literal(row['rank'], datatype=XSD.integer)))
    g.add((ranking_uri, properties_dict_uri['datePosted'], Literal(row['date'],datatype=XSD.date)))

In [50]:
import ast

## add twitter and youtube followers
def convert_to_dict(string):
    
    if string == '[]':
        return {}
    
    dict_string = string
    processed_string = dict_string.replace("Decimal('", "'").replace("')", "'")
    result_dict = ast.literal_eval(processed_string)
    
    for key, values in result_dict.items():
        
        if values is not None:
            result_dict[key] = [value if (value is not None and value.isdigit()) else value for value in values]
    return result_dict

for idx, row in quantity_df.iterrows():
    
    
    artist_uri = URIRef(WD + quote(row['Artist']))
    
    if row['Artist'] == 'Post Malone':
        print(row['mostRecentSocialMediaFollowers'])
    
    if row['mostRecentSocialMediaFollowers'] != '[]':
        social_media_followers = convert_to_dict(row['mostRecentSocialMediaFollowers'])
        for key, values in social_media_followers.items():
            
            if key == 'Twitter/X' and values is not None:
                g.add((artist_uri, properties_dict_uri['hasTwitterFollowers'], Literal(values[1],datatype=XSD.integer)))
            elif key == 'Youtube' and values is not None:
                g.add((artist_uri, properties_dict_uri['hasYoutubeFollowers'], Literal(values[1],datatype=XSD.integer)))


{'Twitter/X': ['P6552', Decimal('7045719')], 'Youtube': ['P2397', Decimal('26400000')]}


In [51]:
g.serialize('KnowledgeGraphs//knowledge_graph.ttl', format='turtle')

<Graph identifier=Nf44a790461fb425396682f633a909df1 (<class 'rdflib.graph.Graph'>)>